In [ ]:
import serial


# Replace 'COM3' with the appropriate port for your Arduino

ser = serial.Serial('COM3', 9600)

while True:
    msg=input("Input")
    if msg=="exit":
        break
    ser.write(msg.encode('utf-8'))
    #value=ser.readline()

In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import pandas as pd
import serial
from keras.models import load_model
from cvzone.FaceDetectionModule import FaceDetector
from cvzone.HandTrackingModule import HandDetector
import os
import cvzone

In [ ]:
mp_drawing= mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

def detection(image, model):
    image =  cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable= False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    return image,results

def draw_landmark(image, results):

    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=3),
                                mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=2)
                                )
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=3),
                                mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=2)
                                )
    
def get_landmarks(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
    right_hand = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
   
    return np.concatenate([pose, right_hand])


actions= np.array(['left','right','up','down'])
no_sequences = 120 # 60 frames as video
sequence_length= 30 # 30fps

label_map = {label:num for num, label in enumerate(actions)}
label_map

model = load_model('action_mod.h5')

Gesture Recognition

In [12]:
sequence = []
sentence = []
predictions = []
threshold = 0.9
hd = HandDetector(detectionCon=threshold)
try:
    S=serial.Serial(port='COM3',baudrate=9600)
except:
    print("Error connecting to Motor Control Unit")

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()
        
        #Find Hands
        hand,imgs = hd.findHands(frame,draw=False)
        
        #Detect Finger count
        if hand:
            lefthand = hand[0]
            bbox = lefthand["bbox"]
            lmlist = lefthand['lmList']
            handtype = lefthand['type']
            fingersup = hd.fingersUp(lefthand)
            totalfingers = fingersup.count(1)
            #print(totalfingers)

        # Make detections
        image, results = detection(frame, holistic)
        #print(results)
        
        # Draw landmarks
        draw_landmark(image,results)

        keypoints = get_landmarks(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            #print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])
            if sentence:
                if len(sentence)==1:
                    data=sentence[-1]
                    try:
                        S.write((str(data)+'\n').encode('UTF-8'))
                    except:
                        print("error")
                    print(data)
                if data!= sentence[-1]:
                    data=sentence[-1]
                    #S.write((str(sentence[-1])+' '+str(totalfingers)+'\r\n').encode('UTF-8'))
                    try:
                        S.write((str(data)+'\n').encode('UTF-8'))
                    except:
                        print("error")
                    print(data)
                    #value=S.readline()
                    #valueInString=str(value,'UTF-8')
                    #time.sleep(1)
                    #print(value.decode('UTF-8'))

            if len(sentence) > 5:
                sentence = sentence[-5:]

            # Viz probabilities
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

Error connecting to Motor Control Unit
1/1 [==============================] - 0s 50ms/step
right
1/1 [==============================] - 0s 52ms/step
right
1/1 [==============================] - 0s 50ms/step
right
1/1 [==============================] - 0s 48ms/step
right
1/1 [==============================] - 0s 57ms/step
right
1/1 [==============================] - 0s 55ms/step
right
1/1 [==============================] - 0s 60ms/step
right
1/1 [==============================] - 0s 69ms/step
left
1/1 [==============================] - 0s 47ms/step
up
1/1 [==============================] - 0s 35ms/step
down
1/1 [==============================] - 0s 46ms/step
right
1/1 [==============================] - 0s 39ms/step
up
1/1 [==============================] - 0s 37ms/step


In [ ]:
cap.release()
cv2.destroyAllWindows()

In [ ]:
import serial


# Replace 'COM3' with the appropriate port for your Arduino
ser = serial.Serial('COM3', 9600)

while True:
    msg=input("Input")
    if msg=="exit":
        break
    ser.write(msg.encode('utf-8'))
    value=ser.readline()
    valueInString=str(value,'UTF-8')
    #time.sleep(1)
    print(value.decode('UTF-8'))
    